# Basic Data Access

Demonstrates opening a case and retrieving component data using indexing.

The primary way to open a case is to instantiate a `GridWorkBench` object. For these examples, a `case_path` variable is created in a hidden cell that reads from `case.txt`. The code below shows the syntax used for initialization.

```python
from esapp import GridWorkBench
from esapp.components import *

wb = GridWorkBench(case_path)
```

In [ ]:
# This cell is hidden in the documentation.
# It performs the actual case loading for the example.
from esapp import GridWorkBench
from esapp.components import *
import numpy as np
import matplotlib.pyplot as plt
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_gen_dispatch_and_voltage, plot_gen_load_balance

## Retrieve Component Data

The primary interface for accessing power system data is through the indexing syntax `wb[ComponentType, fields]`, which returns a pandas DataFrame. This provides a flexible way to extract and analyze component parameters.

In [ ]:
bus_voltages = wb[Bus, "BusPUVolt"]
bus_voltages.head()

Retrieve multiple fields at once and filter the results. Here we get all online generators with their power output:

In [ ]:
gens = wb[Gen, ["GenMW", "GenMVR", "GenStatus"]]
online_gens = gens[gens["GenStatus"] == "Closed"]
online_gens.head()

### Visualizing Generator Output

A bar chart provides a quick overview of the generation dispatch across the system.

In [ ]:
V = wb[Bus, ["BusPUVolt", "BusNum"]]
plot_gen_dispatch_and_voltage(online_gens, V)

### Load vs Generation Balance

In [ ]:
loads = wb[Load, ["LoadMW", "LoadMVR"]]
total_gen = online_gens['GenMW'].sum()
total_load = loads['LoadMW'].sum()

plot_gen_load_balance(total_gen, total_load)

## Modify Component Data

Use the indexing syntax to update component parameters. Setting a field to a scalar value broadcasts it to all components of that type:

In [ ]:
wb[Gen, "GenMW"] = 100.0

## Verify Modification

Read the modified values to confirm the update:

In [ ]:
wb[Gen, "GenMW"].head(10)